In [ ]:
!pip install unsloth datasets transformers accelerate bitsandbytes wandb huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/5

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from huggingface_hub import login
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.9.0+cu130 with CUDA 1300 (you have 2.9.0+cu128)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu130 with CUDA 1300 (you have 2.9.0+cu128)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!


## 🔐 Authenticate: Hugging Face & W&B (using Colab secrets)

We’ll read tokens from **Colab’s `userdata`** store:
- `HGFaceApi` → Hugging Face token (required to push or create repos)
- `wb_token` → Weights & Biases token (optional; we’ll log anonymously if absent)

If you haven’t saved them yet: **Runtime → Run anything** → `from google.colab import userdata; userdata.set('HGFaceApi','...'); userdata.set('wb_token','...')`


In [ ]:
from google.colab import userdata
from huggingface_hub import login
import wandb

# Fetch tokens from Colab secret storage
hf_token = userdata.get("HGFaceApi")
wb_token = userdata.get("wb_token")

# Hugging Face login (required)
if not hf_token:
    raise ValueError("Hugging Face token not found in Colab secrets (key: 'HGFaceApi'). Please set it via userdata.set().")
login(hf_token)

# Weights & Biases login (optional: fall back to anonymous)
if wb_token:
    wandb.login(key=wb_token, relogin=True)
else:
    print("⚠️ W&B token ('wb_token') not found. Proceeding with anonymous logging.")

# Initialize a W&B run
run = wandb.init(
    project="LoRA-Finetuning-SmolLM2-135M",
    job_type="training",
    anonymous="allow"  # ensures run creation even without wb_token
)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shubhamjaysukhbhai-kothiya (shubhamjaysukhbhai-kothiya-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


## ⚙️ Step 4: Initialize SmolLM2-135M for LoRA / QLoRA Fine-Tuning

Next, we’ll load the **SmolLM2-135M** model using **Unsloth** in 4-bit mode.  
This configuration enables *parameter-efficient fine-tuning (LoRA)* by quantizing weights to reduce GPU memory usage while maintaining strong performance.


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True  # LoRA supports 4-bit quantization to save VRAM

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/smollm2-135m",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

## 🧩 Step 5: Attach LoRA Adapters

We’ll wrap the base model with **LoRA** adapters to enable parameter-efficient fine-tuning.  
The attention and MLP projection layers are adapted, while the original weights remain frozen.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

print("✅ LoRA adapters added successfully!")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.2 patched 30 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ LoRA adapters added successfully!


## 📘 Step 6: Load and Format the Alpaca Dataset

We’ll use a small subset (500 examples) of the **Alpaca dataset** for instruction-tuning.  
Each entry is reformatted into a standardized *Instruction–Input–Response* prompt so it matches the format the model expects.


In [ ]:
dataset = load_dataset("tatsu-lab/alpaca", split="train[:500]")

prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for inst, inp, out in zip(instructions, inputs, outputs):
        if inp.strip() == "":
            text = prompt_style.format(inst, "N/A", out) + EOS_TOKEN
        else:
            text = prompt_style.format(inst, inp, out) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)
print("✅ Sample formatted data:\n", dataset["text"][0][:400])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

✅ Sample formatted data:
 Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Input:
N/A

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough s


## 🏋️ Step 7: Begin Training and Monitor GPU Resources

Before starting fine-tuning, we’ll display the active GPU model and total available memory.  
Then we’ll launch the LoRA fine-tuning process using the configured trainer.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1,
        num_train_epochs = 3,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        output_dir = "outputs",
        save_total_limit = 1,
        report_to = "wandb"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Display the GPU model and available VRAM
gpu_info = torch.cuda.get_device_properties(0)
print(f"🖥️ GPU: {gpu_info.name} | Total Memory: {round(gpu_info.total_memory / 1e9, 2)} GB")

# Begin training and store performance metrics
trainer_stats = trainer.train()

print("✅ LoRA fine-tuning completed successfully!")

The model is already on multiple devices. Skipping the move to device specified in `args`.


🖥️ GPU: Tesla T4 | Total Memory: 15.83 GB


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 3 | Total steps = 189
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


Step,Training Loss
5,2.795800
10,2.653000
15,2.293500
20,2.081300
25,1.901400
30,1.555500
35,1.617400
40,1.466000
45,1.275200
50,1.394700


Unsloth: Will smartly offload gradients to save VRAM!
✅ LoRA fine-tuning completed successfully!


In [ ]:
used_mem = round(torch.cuda.max_memory_reserved() / 1e9, 3)
print(f"⏱ Runtime: {round(trainer_stats.metrics['train_runtime']/60, 2)} minutes")
print(f"💾 Peak reserved GPU memory: {used_mem} GB")

⏱ Runtime: 4.6 minutes
💾 Peak reserved GPU memory: 1.095 GB


## 🧠 Step 9: Test the Fine-Tuned Model with a Sample Instruction

We’ll now switch the model into inference mode and test it on a simple instruction:  
*"Write a Python function to check if a number is prime."*  

The model will generate a response formatted in the same style used during fine-tuning.


In [ ]:
from IPython.display import Markdown

FastLanguageModel.for_inference(model)

prompt = prompt_style.format(
    "Write a Python function to check if a number is even.",
    "",
    ""
)

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=150)
Markdown(tokenizer.decode(outputs[0], skip_special_tokens=True))

Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Write a Python function to check if a number is even.

### Input:


### Response:

```python
def even_number(num):
    """
    Check if a number is even.
    """
    if num % 2 == 0:
        return True
    else:
        return False
```


## 💾 Step 10: Save & Push LoRA Model to Hugging Face

We’ll export the fine-tuned model/tokenizer locally and upload them to the Hub.
- Target repo: `shubh7/SmolLM2-135M-Alpaca-LoRA`
- If this namespace isn’t owned by the current token, we’ll push to your own username automatically.


In [ ]:

from huggingface_hub import HfApi

api = HfApi()

local_dir = "SmolLM2-135M-Alpaca-LoRA"
desired_repo_id = "shubh7/SmolLM2-135M-Alpaca-LoRA"  # your target path

# Save locally
model.save_pretrained(local_dir)
tokenizer.save_pretrained(local_dir)

# Resolve the authenticated username
who = api.whoami(token=hf_token)
username = who["name"]

# If token can't write to the desired namespace, switch to your own
namespace, repo_name = desired_repo_id.split("/", 1)
if namespace != username:
    print(f"ℹ️ Using token for '{username}', not '{namespace}'. "
          f"Pushing to '{username}/{repo_name}' instead.")
    repo_id = f"{username}/{repo_name}"
else:
    repo_id = desired_repo_id

# Create (or reuse) repo, then push
api.create_repo(repo_id=repo_id, repo_type="model", private=False, exist_ok=True, token=hf_token)
model.push_to_hub(repo_id, token=hf_token)
tokenizer.push_to_hub(repo_id, token=hf_token)

print(f"✅ LoRA fine-tuned model uploaded to https://huggingface.co/{repo_id}")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   3%|2         |  558kB / 19.6MB            

Saved model to https://huggingface.co/shubh7/SmolLM2-135M-Alpaca-LoRA


README.md: 0.00B [00:00, ?B/s]

✅ LoRA fine-tuned model uploaded to https://huggingface.co/shubh7/SmolLM2-135M-Alpaca-LoRA
